<a href="https://colab.research.google.com/github/genaiconference/Agentic_RAG_Workshop/blob/main/05_traditional_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Traditional RAG

In [ ]:
!git clone https://github.com/genaiconference/Agentic_RAG_Workshop.git

Cloning into 'Agentic_RAG_Workshop'...
remote: Enumerating objects: 339, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 339 (delta 90), reused 61 (delta 61), pack-reused 229 (from 2)
Receiving objects: 100% (339/339), 39.91 MiB | 12.76 MiB/s, done.
Resolving deltas: 100% (156/156), done.
Updating files: 100% (51/51), done.


## Setup and Installations
Install necessary libraries for document processing, data handling, and interacting with Azure Document Intelligence and OpenAI.

In [ ]:
!pip install -r /content/Agentic_RAG_Workshop/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.0/106.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.9/187.9 kB 13.5 MB/s eta

## Load Environment Variables and Initialize Clients
Load environment variables containing API keys and endpoint information, and initialize the Azure Document Intelligence and OpenAI clients.

In [ ]:
import os

os.chdir("/content/Agentic_RAG_Workshop/")

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

load_dotenv()

llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4.1",
    temperature=0,
)

embeddings = OpenAIEmbeddings(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="text-embedding-3-small"
)

## Initialize Langfuse Handler for Tracing

In [ ]:
from langfuse.langchain import CallbackHandler
from langfuse import get_client

os.environ["LANGFUSE_PUBLIC_KEY"] = os.getenv("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.getenv("LANGFUSE_SECRET_KEY")
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"

langfuse = get_client()

# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

langfuse_handler = CallbackHandler()

Langfuse client is authenticated and ready!


## Create Retrievers for all data sources

Use the Retriever Pipeline which does the following:

- Load DI Output
- Generate Parent docs
- Create Child docs
- Generate Summaries
- Generate Question docs
- Create Chroma DB
- Create Ensemble Retriever

In [ ]:
from multivector_utils import create_retriever_pipeline

### Retriever for Insurance Policy Document

In [ ]:
insurance_policy_retriever = create_retriever_pipeline(
    di_results_filename="Insurance_Policy_results.pkl",
    source_file_name="Insurance_Policy",
    vector_db_name="chroma-insurance-policy",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

### Retriever for Leave Policy Document

In [ ]:
leave_policy_retriever = create_retriever_pipeline(
    di_results_filename="Leave_Policy_results.pkl",
    source_file_name="Leave_Policy",
    vector_db_name="chroma-leave-policy",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

### Retriever for Microsoft 2023 & 2024 Documents

In [ ]:
MS23_retriever = create_retriever_pipeline(
    di_results_filename="Microsoft_2023_results.pkl",
    source_file_name="Microsoft_2023",
    vector_db_name="chroma-microsoft-2023",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

In [ ]:
MS24_retriever = create_retriever_pipeline(
    di_results_filename="Microsoft_2024_results.pkl",
    source_file_name="Microsoft_2024",
    vector_db_name="chroma-microsoft-2024",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

### Retriever for Apple 2023 & 2024 Documents

In [ ]:
AP23_retriever = create_retriever_pipeline(
    di_results_filename="Apple_2023_results.pkl",
    source_file_name="Apple_2023",
    vector_db_name="chroma-apple-2023",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

In [ ]:
AP24_retriever = create_retriever_pipeline(
    di_results_filename="Apple_2024_results.pkl",
    source_file_name="Apple_2024",
    vector_db_name="chroma-apple-2024",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

## Traditional RAG

to add a picture

### Define Prompt

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Avoid using generic phrases like "Provide context" or "as per context.

Question: {input}

Context: {context}

Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

## Combine all Retrievers using LOTR - MergerRetriever

**Lord of the Retrievers (LOTR)**, also known as **MergerRetriever**, takes a list of retrievers as input and merges the results of their get_relevant_documents() methods into a single list. The merged results will be a list of documents that are relevant to the query and that have been ranked by the different retrievers.

The MergerRetriever class can be used to improve the accuracy of document retrieval in a number of ways. First, it can combine the results of multiple retrievers, which can help to reduce the risk of bias in the results. Second, it can rank the results of the different retrievers, which can help to ensure that the most relevant documents are returned first.

In [ ]:
from langchain.retrievers import MergerRetriever

# The Lord of the Retrievers will hold the output of all retrievers and can be used as any other retriever on different types of chains.
lotr = MergerRetriever(retrievers=[insurance_policy_retriever,
                                   leave_policy_retriever,
                                   MS23_retriever,
                                   MS24_retriever,
                                   AP23_retriever,
                                   AP24_retriever])

### Create Simple RAG Chain

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Create a chain to combine documents
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

# Create a retrieval chain
rag_chain = create_retrieval_chain(lotr, combine_docs_chain)

### Test Q&A

In [ ]:
from IPython.display import Markdown

question = "I am planning to adopt a kid. How many leaves can I avail and What are the Insurance benefits I can get? What are the latest adoption policies of Indian Government as of 2025?"
response = rag_chain.invoke({"input": question},config={"callbacks": [langfuse_handler]})

display(Markdown(response["answer"]))

If you are planning to adopt a child, you are eligible for 26 weeks of parental leave, which must be availed within one year of the child's adoption. Insurance benefits under your group medical plan include coverage for hospitalization, pre- and post-hospitalization expenses, and newborn coverage from day one (subject to intimation to HR within 15 days), but you should confirm with your HR or TPA for specific adoption-related inclusions. As of 2025, the latest Indian government adoption policies continue to allow adoption by eligible individuals and couples, with leave and insurance benefits as per organizational and statutory norms; for detailed legal procedures and eligibility, refer to the Central Adoption Resource Authority (CARA) guidelines.

In [ ]:
question = "Compare the profits of Microsoft & Apple in 2024"
response = rag_chain.invoke({"input": question}, config={"callbacks": [langfuse_handler]})

display(Markdown(response["answer"]))

In fiscal year 2024, Microsoft reported net income of $88.1 billion, while Apple reported net income of $93.7 billion. Despite Apple having higher net income, Microsoft's operating income was $109.4 billion compared to Apple's $123.2 billion. Both companies remained highly profitable, but Apple had a higher net profit for the year.